<a href="https://colab.research.google.com/github/kishoretvk/AIML/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project
## **Project Objective**

Develop a Generative AI application using a Large Language Model to **automate the Asspect-based Classification and Summarization of Customer Reviews** received by a business. The application will aim to predict review sentiment, aspect-based sentiment and generate summaries of these customer reviews, and will evaluate the performance of the LLM at these tasks through various evaluation schemes.

## **Question 1: Install the Necessary Libraries (4 Marks)**



In [200]:
!pip install llama-cpp-python==0.2.79
#@title Run this cell to setup Unsloth on Colab
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama_cpp_python==0.2.79 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 190.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Non-user install by explicit request
  Created build tracker: /tmp/pip-build-tracker-z6r1bxnk
  Entered build tracker: /tmp/pip-build-tracker-z6r1bxnk
  Created temporary directory: /tmp/pip-install-k99_khk9
  Created temporary directory: /tmp/pip-ephem-wheel-cache-hh1hjv_d
  1 location(s) to search for versions of scikit-build-core:
  * https://pypi.org/simple/scikit-build-core/
  Fetching project page and analyzing links: https://pypi.org/simple/scikit-build-core/
  Getting page https://pypi.org/simple/scikit-build-core/
  Found index url https://pypi.org/simple/
  Looking up "https://pypi.org/simple/scikit-build-core/" in the cache
  Request header has "max_age" as 0, cache bypas

In [19]:
!pip install huggingface-hub==0.23.4


In [20]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-e8_yvkxo/unsloth_dbb7d9c12e5f4232b89a27bfe6131352
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-e8_yvkxo/unsloth_dbb7d9c12e5f4232b89a27bfe6131352
  Resolved https://github.com/unslothai/unsloth.git to commit 4e570be9ae4ced8cdc64e498125708e34942befc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [21]:
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes


In [22]:
!pip install evaluate==0.4.2 rouge-score==0.1.2 bert-score==0.3.13


In [3]:
!pip install numpy==1.25.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 88.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.0 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.25.2 which is incompatible.
xformers 0.0.26.post1 requires torch==2.3.0, but you have torch 2.3.1+cu121 which is incompatible.


In [7]:
!pip show numpy

Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are
        met:
        
            * Redistributions of source code must retain the above copyright
               notice, this list of conditions and the following disclaimer.
        
            * Redistributions in binary form must reproduce the above
               copyright notice, this list of conditions and the following
               disclaimer in the documentation and/or other materials provided
               with the distribution.
        
            * Neither the name of the NumPy Developers nor the names of any
               contributors may be used to endorse or promo

In [23]:
!pip install torchvision

##**Question 2: Import Libraries

In [201]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate
from datasets import load_dataset

import datasets
from google.colab import drive
import locale

# import huggingface hub and llama cpp

In [203]:
## Import Hf_hub_download from hugging_face_hub
## Import Llama from llama_cpp
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [204]:
# Import FastLanguageModel from unsloth
# Import SFTTrainer from trl
# Import TrainingArguments from transformers

from unsloth import FastLanguageModel

from trl import SFTTrainer
from trl import DPOTrainer


from transformers import TrainingArguments

In [205]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


In [206]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [207]:
import torch

from datasets import load_dataset

from unsloth import PatchDPOTrainer
from unsloth import FastLanguageModel

from transformers import TrainingArguments
from trl import DPOTrainer

In [208]:
# Import FastLanguageModel from unsloth
# Import SFTTrainer from trl
# Import TrainingArguments from transformers

from unsloth import FastLanguageModel

from trl import SFTTrainer
from trl import DPOTrainer


from transformers import TrainingArguments

##**Question 3: Model Setup (4 Marks)**

HuggingFace Link: The Bloke's Llama 2 13B Chat GGUF File

https://huggingface.co/TheBloke/Llama-2-13B-GGUF/blob/main/llama-2-13b.Q5_K_M.gguf


In [214]:
# Add Bloke Version of llama 13B Model from Hugging Face in GGUF Format Here. Also use Model Base Name Keeping in mind the 15 GB maximum GPU Ram available
model_name_or_path = "TheBloke/Llama-2-13B-GGUF"
model_basename = "llama-2-13b.Q5_K_M.gguf"  # the model is in gguf format from Hugging face


In [215]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

In [245]:
import gc

# Function to clear GPU memory
def clear_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# Clear GPU memory
clear_gpu_memory()

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=128,  # Reduced batch size
    n_gpu_layers=4,  # Reduced number of GPU layers
    n_ctx=4096,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-GGUF/snapshots/b106d1c018ac999af9130b83134fb6b7c5331dea/llama-2-13b.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32 

#*Sentiment Classification*

##**Question 4: Data Preprocessing for Sentiment Classification (5 Marks)**(A) Load a CSV File containing the Dataset of 30 Reviews (2 Marks)

(B) Divide the Dataset into Train and Test Dataset (Gold Reviews) such that Train and Test Dataset contains equal number of positive and negative reviews (3 Marks)


In [224]:
# Load a CSV File containing Dataset of 30 Reviews
sample_reviews_df = pd.read_csv("/content/data/customer_reviews_dataset.csv")

In [ ]:
print(sample_reviews_df.head())

  customer_id  customer_name        product_name product_type review_date  \
0      CID041    Aisha Patel    Dell Inspiron 15       Laptop  01/01/2024   
1      CID011   Liam Thomson      JBL Tune 500BT   Headphones  01/05/2024   
2      CID034   Maria García    Mi Power Bank 3i   Power Bank  01/10/2023   
3      CID032  Jamal Johnson  Samsung Galaxy S21   Smartphone  03/03/2023   
4      CID051   Emily Nguyen      JBL Tune 500BT   Headphones  05/25/2023   

   rating review_sentiment                                        review_text  \
0       5         Positive  I bought this laptop for my son who is studyin...   
1       1         Negative  I was very disappointed with these headphones....   
2       4         Positive  Awesome power bank, it charges my phone very f...   
3       2         Negative  I bought this phone mainly for its much-hyped ...   
4       5         Positive  I love these headphones, they are amazing. The...   

                                      aspects_revi

In [ ]:
# Separate positive and negative reviews
positive_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'].str.lower() == 'positive']
negative_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'].str.lower() == 'negative']

# Sample 5 positive and 5 negative reviews for gold examples
positive_gold_examples = positive_reviews.sample(5, random_state=40)
negative_gold_examples = negative_reviews.sample(5, random_state=40)

# Concatenate positive and negative gold examples
sample_reviews_gold_examples_df = pd.concat([positive_gold_examples, negative_gold_examples])

# Create the training set by excluding gold examples
sample_reviews_examples_df = sample_reviews_df.drop(index=sample_reviews_gold_examples_df.index)

# Convert gold examples to JSON
columns_to_select = ['review_text', 'review_sentiment']
gold_examples_json = sample_reviews_gold_examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(gold_examples_json)[0])

# Print the shapes of the datasets
print("Training Set Shape:", sample_reviews_examples_df.shape)
print("Gold Examples Shape:", sample_reviews_gold_examples_df.shape)


{'review_text': 'I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.', 'review_sentiment': 'Positive'}
Training Set Shape: (20, 11)
Gold Examples Shape: (10, 11)


In [ ]:
print(json.loads(gold_examples_json))

[{'review_text': 'I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.', 'review_sentiment': 'Positive'}, {'review_text': 'The laptop is good enough for the price. It has a good performance and a good storage. The screen is bright and the keyboard is good. The laptop is a bit heavy and bulky, but it is not a big deal. The battery life is above average, but the charger is a bit flimsy. I would recommend it for basic tasks and daily use.', 'review_sentiment': 'Positive'}, {'review_text': "Originally bought it for my work, quite happy with it so far! Fast, reliable, easy to use and has a good webcam. Display is good and battery backup is also great. The keyboard is a joy to type on, gives me the old typewriter vibes! Quickly bec

In [ ]:
print(sample_reviews_examples_df)

   customer_id      customer_name        product_name product_type  \
2       CID034       Maria García    Mi Power Bank 3i   Power Bank   
3       CID032      Jamal Johnson  Samsung Galaxy S21   Smartphone   
4       CID051       Emily Nguyen      JBL Tune 500BT   Headphones   
5       CID012   Rafael Fernandez    Mi Power Bank 3i   Power Bank   
7       CID063        Michael Lee    Dell Inspiron 15       Laptop   
8       CID029   Olivia Rodriguez      JBL Tune 500BT   Headphones   
10      CID007          David Kim    Dell Inspiron 15       Laptop   
11      CID040        Reena Gupta    Mi Power Bank 3i   Power Bank   
12      CID070    Emre Çalhanoğlu  Samsung Galaxy S21   Smartphone   
13      CID060      Ketan Kopikar      JBL Tune 500BT   Headphones   
14      CID004    Bradley Wiggins    Dell Inspiron 15       Laptop   
15      CID100     Milad Hosseini    Mi Power Bank 3i   Power Bank   
16      CID030       Rachel Smith  Samsung Galaxy S21   Smartphone   
17      CID010  Mior

In [ ]:
print(sample_reviews_gold_examples_df)

   customer_id      customer_name      product_name product_type review_date  \
0       CID041        Aisha Patel  Dell Inspiron 15       Laptop  01/01/2024   
26      CID067  Adewale Akinfenwa  Dell Inspiron 15       Laptop  01/09/2024   
27      CID053         Pooja Jain  Dell Inspiron 15       Laptop  12/31/2023   
28      CID082    Laura Wolvaardt  Mi Power Bank 3i   Power Bank  04/13/2023   
9       CID080        Karan Singh    JBL Tune 500BT   Headphones  02/22/2023   
1       CID011       Liam Thomson    JBL Tune 500BT   Headphones  01/05/2024   
22      CID076          Amit Shah  Mi Power Bank 3i   Power Bank  03/02/2023   
25      CID013        Ollie Stone    JBL Tune 500BT   Headphones  12/09/2023   
29      CID099       Jim Kowalski  Dell Inspiron 15       Laptop  08/03/2023   
6       CID043         Sofia Chen  Dell Inspiron 15       Laptop  05/20/2023   

    rating review_sentiment  \
0        5         Positive   
26       4         Positive   
27       4         Positiv

In [ ]:
gold_examples = (
        sample_reviews_gold_examples_df.loc[:, columns_to_select]
                                     .sample(10, random_state=40) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [ ]:
print(gold_examples)

[{"review_text":"These are the best headphones I have ever used. The sound quality is amazing, the bass is powerful, and the treble is crisp. The headphones are also very comfortable to wear, they are soft and adjustable. The Bluetooth connection is also very reliable, it does not break or lag. Charging  takes long though (4-5 hrs), but due to a long battery backup the situation's not as bad as it could've been. The headphones also have a handy button that lets me control the music and the calls. I highly recommend these headphones.","review_sentiment":"Positive"},{"review_text":"Good backup for my phone & other devices. Small and light, with two USB-C and two USB-A ports, and a nice LED indicator showing the charge remaining inside it. Good battery capacity. Design is pretty decent, and a fast charging speed makes it even better. Overall, a good purchase for the price I paid.","review_sentiment":"Positive"},{"review_text":"I thought this would be the laptop I could use for my research

##**Question 5: Zero-shot Prompting for Sentiment Classification (8 Marks)**
(A) Write a system message for the model to classify the reviews into positive and negative (4 Marks)

(B) Calculate the Accuracy of the Model by comparing with the Actual Result (2 Marks)

(C) Evaluate on Gold Reviews (2 Marks)

In [ ]:
sample_reviews_examples_df


,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
3,CID032,Jamal Johnson,Samsung Galaxy S21,Smartphone,03/03/2023,2,Negative,I bought this phone mainly for its much-hyped ...,"{ display : Positive , camera : Negative , Bat...",I'm truly sorry to hear that the camera perfor...,The user expressed disappointment with the pho...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...
5,CID012,Rafael Fernandez,Mi Power Bank 3i,Power Bank,01/13/2024,1,Negative,"This is a very bad power bank, it does not cha...","{ battery : Negative , charging : Negative }",I'm deeply concerned to hear about your experi...,"The customer reviews a poor power bank, statin..."
7,CID063,Michael Lee,Dell Inspiron 15,Laptop,02/09/2023,2,Negative,This is a below average laptop. It has inconsi...,"{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear that your laptop isn't meeti...,The customer reviews a below-average laptop wi...
8,CID029,Olivia Rodriguez,JBL Tune 500BT,Headphones,01/02/2024,1,Negative,"I bought these headphones a week ago, and they...","{ sound : Negative , comfort : Negative , char...",I apologize for the issues you're experiencing...,The customer purchased poor-quality headphones...
10,CID007,David Kim,Dell Inspiron 15,Laptop,05/03/2023,1,Negative,"I hate this laptop, it is a nightmare. I bough...","{ battery : Negative , keyboard : Negative , d...",I'm truly sorry to hear about the difficulties...,The customer reviews a laptop with poor softwa...
11,CID040,Reena Gupta,Mi Power Bank 3i,Power Bank,10/02/2023,5,Positive,Recently tried a power bank sharing service - ...,"{ battery : Positive , charging : Negative }",Thank you for your feedback on the power bank ...,The customer reviews a power bank sharing serv...
12,CID070,Emre Çalhanoğlu,Samsung Galaxy S21,Smartphone,11/03/2023,2,Negative,"I am very unhappy with this phone, it has a ve...","{ display : Negative , camera : Negative , Bat...",I'm truly sorry to hear about your dissatisfac...,The customer expresses dissatisfaction with th...
13,CID060,Ketan Kopikar,JBL Tune 500BT,Headphones,01/06/2024,2,Negative,"I bought these headphones just 2 weeks ago, an...","{ sound : Negative , comfort : Negative , char...",I'm sorry to hear about your negative experien...,"The customer is unhappy with their headphones,..."


In [ ]:
sample_reviews_examples_df.review_sentiment.value_counts()

,count
review_sentiment,
Positive,10
Negative,10


In [ ]:
def generate_llama_response(system_message, input_text, temp):
    # Combine system_message and input_text to create the prompt
    prompt = f"[INST]{system_message}\n{input_text}[/INST]"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=temp,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    #print(response)
    return response_text

###**(A) Write System Message Here**

In [ ]:
system_message_zero_shot = """
<<SYS>>You are a helpful and harmless AI assistant.
 You are a sentiment analysis expert.
 You will be provided with customer reviews.
 Analyze each review and determine the sentiment expressed as either positive or negative.
<</SYS>>
"""

In [ ]:
user_message_template = "```{review}```"

In [ ]:
input_text = """
I am extremely disappointed with the service I received at your store!
The staff was rude and unhelpful, showing no regard for my concerns.
Not only did they ignore my requests for assistance, but they also had the audacity to speak to me condescendingly.
It's clear that your company values profit over customer satisfaction.
I will never shop here again and will make sure to spread the word about my awful experience.
You've lost a loyal customer, and I hope others steer clear of your establishment!
"""

In [ ]:
# Combine the user message template with the input text
user_message = user_message_template.format(review=input_text)


In [ ]:
# Generate the response
temperature = 0.4
response = generate_llama_response(system_message_zero_shot, user_message, temperature)
print("result below ")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2069.13 ms
llama_print_timings:      sample time =       1.26 ms /     2 runs   (    0.63 ms per token,  1589.83 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1002.15 ms /     2 runs   (  501.08 ms per token,     2.00 tokens per second)
llama_print_timings:       total time =    1005.86 ms /     2 tokens


result below 




In [ ]:
# def generate_llama_response(system_message, input_text, temp):
#     # Combine system_message and input_text to create the prompt
#     prompt = f"[INST]{system_message}\n{input_text}[/INST]"
#     #print("Prompt:", prompt)  # Debugging: Print the prompt

#     # Generate a response from the LLaMA model
#     response = lcpp_llm(
#         prompt=prompt,
#         max_tokens=512,  # Increased max_tokens
#         temperature=temp,
#         top_p=0.95,
#         repeat_penalty=1.2,
#         top_k=50,
#         stop=['INST'],
#         echo=False
#     )
#     print("Response:", response)  # Debugging: Print the raw response

#     # Extract and return the response text
#     if "choices" in response and len(response["choices"]) > 0:
#         response_text = response["choices"][0]["text"]
#         return response_text
#     else:
#         return "No response generated."

# # Define the system message, user message template, and input text
# system_message_zero_shot = """
# <<SYS>>You are a helpful and harmless AI assistant.
#  You are a sentiment analysis expert.
#  You will be provided with customer reviews.
#  Analyze each review and determine the sentiment expressed as either positive or negative.
# <</SYS>>
# """
# user_message_template = "```{review}```"
# input_text = """
# I am extremely disappointed with the service I received at your store!
# The staff was rude and unhelpful, showing no regard for my concerns.
# Not only did they ignore my requests for assistance, but they also had the audacity to speak to me condescendingly.
# It's clear that your company values profit over customer satisfaction.
# I will never shop here again and will make sure to spread the word about my awful experience.
# You've lost a loyal customer, and I hope others steer clear of your establishment!
# """

# # Combine the user message template with the input text
# user_message = user_message_template.format(review=input_text)

# # Generate the response
# temperature = 0.8  # Slightly adjusted temperature
# response = generate_llama_response(system_message_zero_shot, user_message, temperature)
# print(response)

In [ ]:
print(response)

##*Measuring Prompt Performance*

1.   List item
2.   List item



In [ ]:
sample_reviews = sample_reviews_examples_df.review_text.values

In [ ]:
sentiment_predictions = []

In [ ]:
generate_llama_response( system_message_zero_shot , input_text , 0.2 )

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2069.13 ms
llama_print_timings:      sample time =      22.86 ms /    37 runs   (    0.62 ms per token,  1618.48 tokens per second)
llama_print_timings: prompt eval time =    1912.63 ms /   119 tokens (   16.07 ms per token,    62.22 tokens per second)
llama_print_timings:        eval time =   18004.60 ms /    36 runs   (  500.13 ms per token,     2.00 tokens per second)
llama_print_timings:       total time =   19967.51 ms /   155 tokens


'\n\n\n<<SYS>>\n You have analyzed 1 reviews.\n Your accuracy is: 0% (0 / 1)\n<</SYS>>'

In [ ]:
for sample_review in tqdm(sample_reviews):
    try:
        sentiment_predictions.append(generate_llama_response( system_message_zero_shot , sample_review , 0.1 ))
    except Exception as e:
        print(e)
        sentiment_predictions.append("")

  0%|          | 0/20 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    2069.13 ms
llama_print_timings:      sample time =      76.32 ms /   126 runs   (    0.61 ms per token,  1651.03 tokens per second)
llama_print_timings: prompt eval time =    1857.56 ms /    91 tokens (   20.41 ms per token,    48.99 tokens per second)
llama_print_timings:        eval time =   62558.37 ms /   125 runs   (  500.47 ms per token,     2.00 tokens per second)
llama_print_timings:       total time =   64590.41 ms /   216 tokens
  5%|▌         | 1/20 [02:25<46:00, 145.29s/it]


KeyboardInterrupt: 

In [ ]:
sample_reviews_examples_df["sentiment_prediction"] = sentiment_predictions

In [ ]:
sample_reviews_examples_df.sentiment_prediction.value_counts()

In [ ]:
sample_reviews_examples_df

In [ ]:
sample_reviews_examples_df['sentiment_prediction_cleaned'] = sample_reviews_examples_df['sentiment_prediction'].str.extract(r'(Positive|Negative)', flags=re.IGNORECASE)

In [ ]:
sample_reviews_examples_df['sentiment_prediction_cleaned'] = sample_reviews_examples_df['sentiment_prediction_cleaned'].apply(lambda x: str(x).lower())

In [ ]:
sample_reviews_examples_df

 ### **(B) Calculate Accuracy Here**

In [ ]:
sample_reviews_examples_df['sentiment_prediction_cleaned'] = sample_reviews_examples_df['sentiment_prediction_cleaned'].str.lower()
sample_reviews_examples_df['review_sentiment'] = sample_reviews_examples_df['review_sentiment'].str.lower()

# Compare the predicted labels to the actual labels (case insensitive)
correct_predictions = sample_reviews_examples_df['sentiment_prediction_cleaned'] == sample_reviews_examples_df['review_sentiment']

# Calculate the accuracy
accuracy = correct_predictions.mean()

print(correct_predictions)
print(accuracy)

In [ ]:
accuracy_zero_shot = correct_predictions / len(sample_reviews_examples_df)


In [ ]:
accuracy_zero_shot

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for movie reviews

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths = [], []

    for example in json.loads(gold_examples):
        gold_input = example['review_text']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(review=gold_input)
            }
        ]

        try:
            prediction = generate_llama_response( prompt , user_input , 0.8 )
            print("prediction : " + prediction + "\n")

            prediction_extracted = re.search(r'(Positive|Negative)', prediction, flags=re.IGNORECASE).group().lower()
            # sentiment = match.group().lower() if match else "Sentiment not found."
            print("model_prediction : " + prediction_extracted + "\n")

            model_predictions.append(prediction_extracted) # <- removes extraneous white space and lowercases output
            ground_truths.append(example['review_sentiment'].lower())

            print("ground truth : " + example['review_sentiment'].lower() + "\n")

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    return micro_f1_score

In [ ]:
# evaluate_prompt(system_message_zero_shot, gold_examples, user_message_template)

### **(C) Evaluate the Prompt Here**

In [ ]:
evaluate_prompt(system_message_zero_shot, gold_examples, user_message_template)

##**Question 6: Few-shot Prompting for Sentiment Classification (8 Marks)**

> Add blockquote

(A) Write a system message for the model to classify the reviews into positive and negative. (4 Marks)

(B) Calculate the Accuracy of the Model by comparing with the Actual Result (2 Marks)

(C) Evaluate on Gold Reviews (2 Marks)

In [ ]:
sample_reviews_examples_df.review_sentiment.value_counts()

### **(A) Write a system message for the model to classify the reviews into positive and negative**


In [ ]:
system_message_few_shot = """
<<SYS>>You are a helpful and harmless AI assistant.
You are a sentiment analysis expert.
You will be provided with customer reviews.
Analyze each review and determine the sentiment expressed as either positive or negative.

Here are some examples to guide you:

Review: "I am extremely disappointed with the service I received at your store! The staff was rude and unhelpful, showing no regard for my concerns. Not only did they ignore my requests for assistance, but they also had the audacity to speak to me condescendingly. It's clear that your company values profit over customer satisfaction. I will never shop here again and will make sure to spread the word about my awful experience. You've lost a loyal customer, and I hope others steer clear of your establishment!"
Sentiment: negative

Review: "I had a wonderful experience at your store! The staff was incredibly friendly and went out of their way to help me find what I needed. I was impressed by their knowledge and willingness to assist. I will definitely be coming back and recommending your store to my friends and family. Thank you for the excellent service!"
Sentiment: positive

Review: "The product quality is decent, but the customer service could use some improvement. I had to wait a long time before someone attended to me, and even then, they seemed uninterested in helping. Overall, it was an average experience."
Sentiment: neutral

Now, analyze the following review and determine its sentiment:
```{review}```
<</SYS>>
"""

In [ ]:
def create_examples_with_seed(dataset, n=2, random_seed=None):
    """
    Return two DataFrames with randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (text + label)
        n (int): number of examples of each class to be selected
        random_seed (int): seed for reproducibility (default is None)

    Output:
        few_shot_examples_df (DataFrame): A DataFrame with examples in random order
        new_df (DataFrame): A new DataFrame excluding selected examples
    """

    positive_reviews = (dataset.review_sentiment == 'positive')
    negative_reviews = (dataset.review_sentiment == 'negative')
    columns_to_select = ['review_text', 'review_sentiment']

    # Set a fixed random seed for reproducibility
    np.random.seed(random_seed)

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    few_shot_examples_df = pd.concat([positive_examples, negative_examples])
    # sampling without replacement is equivalent to random shuffling
    few_shot_examples_df = few_shot_examples_df.sample(2 * n, replace=False)

    # Create a new DataFrame excluding selected examples
    new_df = dataset.drop(index=few_shot_examples_df.index)

    return few_shot_examples_df, new_df


In [ ]:
user_message_template = "```{review}```"

In [ ]:
def generate_llama_response( system_message ,  few_shot_examples  , new_review , temp ):

    # Combine user_prompt and system_message to create the prompt
    prompt = f"[INST]{system_message}\n{few_shot_examples}\n{'user'}: {user_message_template.format(review=new_review)}[/INST]"


    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=temp,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text



**Measuring Prompt Performance**

In [ ]:
accuracy_list = []

### **(B) Calculate Accuracy and Mean Accuracy Here**

In [ ]:
for i in range(5):
    few_shot_examples_df, sample_reviews_df = create_examples_with_seed(sample_reviews_examples_df, n=2, random_seed=i)

    few_shot_examples = [
        {
            "review": row["review_text"],
            "sentiment": row["review_sentiment"].lower()
        }
        for _, row in few_shot_examples_df.iterrows()
    ]

    few_shot_examples_str = json.dumps(few_shot_examples)
    sample_reviews = sample_reviews_df.review_text.values

    sentiment_predictions = []

    for sample_review in tqdm(sample_reviews):
        try:
            response = generate_llama_response(system_message_few_shot, few_shot_examples_str, sample_review, 0.6)
            sentiment_predictions.append(response)
        except Exception as e:
            print(e)
            sentiment_predictions.append("")  # Default value for failed predictions

    sentiment_groundtruth = sample_reviews_df.review_sentiment.str.lower().values

    # Cleaning and validating predictions
    sentiment_predictions_cleaned = []
    for prediction in sentiment_predictions:
        if prediction:  # Check if prediction is not empty
            match = re.search(r'(positive|negative)', prediction, flags=re.IGNORECASE)
            if match:
                sentiment_predictions_cleaned.append(match.group().lower())
            else:
                sentiment_predictions_cleaned.append("unknown")  # Or any other default value for unmatched patterns
        else:
            sentiment_predictions_cleaned.append("unknown")  # Or any other default value for empty predictions

    # Calculate Accuracy Here:
    correct_predictions = sum([1 for pred, true in zip(sentiment_predictions_cleaned, sentiment_groundtruth) if pred == true])
    accuracy = correct_predictions / len(sentiment_groundtruth)

    # Append Calculated Accuracy to Accuracy List
    accuracy_list.append(accuracy)


In [ ]:
accuracy_list

In [ ]:
mean_accuracy = np.mean(accuracy_list)
print(f"Mean Accuracy: {mean_accuracy}")

### **(C) Evaluation Few Shot Prompting**

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for movie reviews

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths = [], []

    for example in json.loads(gold_examples):
        gold_input = example['review_text']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(review=gold_input)
            }
        ]

        try:
            prediction = generate_llama_response( prompt ,few_shot_examples, user_input , 0.1 )
            print("prediction : " + prediction + "\n")

            prediction_extracted = re.search(r'(Positive|Negative)', prediction, flags=re.IGNORECASE).group().lower()
            # sentiment = match.group().lower() if match else "Sentiment not found."
            print("model_prediction : " + prediction_extracted + "\n")

            model_predictions.append(prediction_extracted) # <- removes extraneous white space and lowercases output
            ground_truths.append(example['review_sentiment'].lower())

            print("ground truth : " + example['review_sentiment'].lower() + "\n")

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    return micro_f1_score

### **(C) Evaluate Prompt Here**

In [ ]:
# Evaluate the prompt
micro_f1_score = evaluate_prompt(system_message_few_shot, gold_examples, user_message_template)
print(f"Micro-F1 Score: {micro_f1_score}")

#*Aspect-based Sentiment Classification*

##*Data Preprocessing*

##**Question 7: Data Preprocessing for Aspect Based Sentiment Analysis (5 Marks)**

(A) Read the CSV File (2 Marks)

(B) Divide the Dataset Based on product_type into 22 and Train examples and 8 Test examples (Gold Reviews), such that Test Data contains 2 examples of each of the 4 product_type (3 Marks)

In [149]:
sample_reviews_df = pd.read_csv("/content/data/customer_reviews_dataset.csv")

In [38]:
sample_reviews_df

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
0,CID041,Aisha Patel,Dell Inspiron 15,Laptop,01/01/2024,5,Positive,I bought this laptop for my son who is studyin...,"{ battery : Positive , keyboard : Positive, di...",It's fantastic to hear that the laptop you pur...,"The user purchased a laptop for their son, who..."
1,CID011,Liam Thomson,JBL Tune 500BT,Headphones,01/05/2024,1,Negative,I was very disappointed with these headphones....,"{ sound : Negative , comfort : Negative , char...",I'm truly sorry to hear about your disappointi...,The user expressed disappointment with poor so...
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
3,CID032,Jamal Johnson,Samsung Galaxy S21,Smartphone,03/03/2023,2,Negative,I bought this phone mainly for its much-hyped ...,"{ display : Positive , camera : Negative , Bat...",I'm truly sorry to hear that the camera perfor...,The user expressed disappointment with the pho...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...
5,CID012,Rafael Fernandez,Mi Power Bank 3i,Power Bank,01/13/2024,1,Negative,"This is a very bad power bank, it does not cha...","{ battery : Negative , charging : Negative }",I'm deeply concerned to hear about your experi...,"The customer reviews a poor power bank, statin..."
6,CID043,Sofia Chen,Dell Inspiron 15,Laptop,05/20/2023,1,Negative,"I bought this laptop a month ago, and it's alr...","{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear about your laptop issues and...,"The customer reports issues with their laptop,..."
7,CID063,Michael Lee,Dell Inspiron 15,Laptop,02/09/2023,2,Negative,This is a below average laptop. It has inconsi...,"{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear that your laptop isn't meeti...,The customer reviews a below-average laptop wi...
8,CID029,Olivia Rodriguez,JBL Tune 500BT,Headphones,01/02/2024,1,Negative,"I bought these headphones a week ago, and they...","{ sound : Negative , comfort : Negative , char...",I apologize for the issues you're experiencing...,The customer purchased poor-quality headphones...
9,CID080,Karan Singh,JBL Tune 500BT,Headphones,02/22/2023,5,Positive,These are the best headphones I have ever used...,"{ sound : Positive , comfort : Positive , char...",Thank you for sharing your positive experience...,The customer praises the headphones for their ...


In [150]:
laptop_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Laptop']
headphones_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Headphones']
smartphone_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Smartphone']
power_bank_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Power Bank']


laptop_gold_examples = laptop_reviews.sample(2, random_state=40)
headphones_gold_examples = headphones_reviews.sample(2, random_state=40)
smartphone_gold_examples = smartphone_reviews.sample(2, random_state=40)
power_bank_gold_examples = power_bank_reviews.sample(2, random_state=40)



# Concatenate positive and negative gold examples
sample_reviews_gold_examples_df = pd.concat([laptop_gold_examples, headphones_gold_examples,smartphone_gold_examples,power_bank_gold_examples])

# Create the training set by excluding gold examples
sample_reviews_examples_df = sample_reviews_df.drop(index=sample_reviews_gold_examples_df.index)

# Convert gold examples to JSON
columns_to_select = ['review_text', 'product_type','aspects_review']
gold_examples_json = sample_reviews_gold_examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(gold_examples_json)[0])

# Print the shapes of the datasets
print("Training Set Shape:", sample_reviews_examples_df.shape)
print("Gold Examples Shape:", sample_reviews_gold_examples_df.shape)


{'review_text': "Originally bought it for my work, quite happy with it so far! Fast, reliable, easy to use and has a good webcam. Display is good and battery backup is also great. The keyboard is a joy to type on, gives me the old typewriter vibes! Quickly become my main laptop for everyday use, and I'm very satisfied with my purchase.", 'product_type': 'Laptop', 'aspects_review': '{ battery : Positive , keyboard : Positive , display : Positive }'}
Training Set Shape: (22, 11)
Gold Examples Shape: (8, 11)


In [151]:
sample_reviews_gold_examples_df

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
27,CID053,Pooja Jain,Dell Inspiron 15,Laptop,12/31/2023,4,Positive,"Originally bought it for my work, quite happy ...","{ battery : Positive , keyboard : Positive , d...",Thank you for sharing your positive experience...,The customer is satisfied with their work lapt...
14,CID004,Bradley Wiggins,Dell Inspiron 15,Laptop,05/10/2024,5,Positive,"This is a great laptop, I am very happy with i...","{ battery : Positive , keyboard : Positive , d...",Thank you for your positive feedback on our la...,"The customer is satisfied with their laptop, s..."
23,CID075,Jack Ryder,JBL Tune 500BT,Headphones,05/05/2023,5,Positive,Great for listening to music or podcasts. Good...,"{ sound : Positive , comfort : Positive , char...",Thank you for your positive review of our head...,Customer praises headphones for good sound qua...
13,CID060,Ketan Kopikar,JBL Tune 500BT,Headphones,01/06/2024,2,Negative,"I bought these headphones just 2 weeks ago, an...","{ sound : Negative , comfort : Negative , char...",I'm sorry to hear about your negative experien...,"The customer is unhappy with their headphones,..."
16,CID030,Rachel Smith,Samsung Galaxy S21,Smartphone,08/28/2023,4,Positive,"Good phone, I am satisfied with it. The phone ...","{ display : Positive , camera : Positive , Bat...",Thank you for your positive review of the phon...,"The customer is satisfied with their phone, pr..."
3,CID032,Jamal Johnson,Samsung Galaxy S21,Smartphone,03/03/2023,2,Negative,I bought this phone mainly for its much-hyped ...,"{ display : Positive , camera : Negative , Bat...",I'm truly sorry to hear that the camera perfor...,The user expressed disappointment with the pho...
24,CID046,Harvey Stark,Mi Power Bank 3i,Power Bank,07/19/2023,5,Positive,I love this power bank - it's amazing. It can ...,"{ battery : Positive , charging : Positive }",Thank you for your enthusiastic feedback on ou...,The customer praises the power bank's capacity...
5,CID012,Rafael Fernandez,Mi Power Bank 3i,Power Bank,01/13/2024,1,Negative,"This is a very bad power bank, it does not cha...","{ battery : Negative , charging : Negative }",I'm deeply concerned to hear about your experi...,"The customer reviews a poor power bank, statin..."


In [152]:
gold_examples = (
        sample_reviews_gold_examples_df.loc[:, columns_to_select]
                                     .sample(8, random_state=40) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [153]:
user_message_template = """{review}"""

## **Question 8: Zero-shot Prompting for ABSA (8 Marks)**
(A) Write System Prompt to get the Aspect Based Sentiment for each of the review in format mentioned in csv. You will need to first identify the product , then based on the product , you will know the aspects for which you need sentiments (4 Marks)

(B) Find Accuracy by comparing to actual predictions (2 Marks)

(C) Evaluate the model (2 Marks)

###**(A) Write your Prompt Here**

In [169]:
zero_shot_system_message = """
<<SYS>>You are an AI Assistant specialized in aspect-based sentiment analysis of customer reviews. For each input text, you will:
1. Identify the relevant product aspects mentioned in the review.
2. Determine the sentiment for each identified product aspect.

Ensure that the sentiments are clearly labeled as 'Positive', 'Negative', or 'Neutral' for each aspect.

### Example:
Input Review: "I bought a laptop. Its performance is good, but the battery backup is not so good."
Output: {"performance": "Positive", "battery backup": "Negative"}

### Instructions:
1. **Identify Product Aspects**: Extract the key aspects of the product mentioned in the review.
2. **Analyze Sentiment**: Determine the sentiment (Positive, Negative, or Neutral) for each identified aspect.
3. **Output Format**: Provide the output in JSON format with aspect-sentiment pairs.

<</SYS>>
"""

In [170]:
def generate_llama_response(system_message, input_text, temp):
    # Combine system_message and input_text to create the prompt
    prompt = f"{system_message}\nInput Review: {input_text}\nOutput:"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=temp,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['<</SYS>>'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"].strip()
    return response_text

In [45]:
input_text = """
I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design.
The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
"""

In [171]:
response = generate_llama_response(zero_shot_system_message, input_text, 0.8)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2612.94 ms
llama_print_timings:      sample time =      11.53 ms /    20 runs   (    0.58 ms per token,  1734.61 tokens per second)
llama_print_timings: prompt eval time =    2508.71 ms /   327 tokens (    7.67 ms per token,   130.35 tokens per second)
llama_print_timings:        eval time =    9451.59 ms /    19 runs   (  497.45 ms per token,     2.01 tokens per second)
llama_print_timings:       total time =   11986.13 ms /   346 tokens


In [172]:
response

'{"keyboard": "Positive", "performance": "Negative"}'

###*Measuring Prompt Performance*

In [52]:
sample_reviews = sample_reviews_examples_df.review_text.values

In [53]:
sentiment_predictions = []

In [54]:
generate_llama_response( zero_shot_system_message , input_text , 0.8 )

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2612.94 ms
llama_print_timings:      sample time =      22.07 ms /    36 runs   (    0.61 ms per token,  1631.47 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   19131.33 ms /    36 runs   (  531.43 ms per token,     1.88 tokens per second)
llama_print_timings:       total time =   19179.58 ms /    36 tokens


'{“keyboard”: “Positive”, “battery life”: “Positive”, “antivirus software”: “Neutral”}'

In [55]:
for sample_review in tqdm(sample_reviews):
    try:
        sentiment_predictions.append(generate_llama_response( zero_shot_system_message , sample_review , 0.6 ))
    except Exception as e:
        print(e)
        sentiment_predictions.append("")

  0%|          | 0/22 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    2612.94 ms
llama_print_timings:      sample time =      15.95 ms /    26 runs   (    0.61 ms per token,  1629.89 tokens per second)
llama_print_timings: prompt eval time =    1924.90 ms /    77 tokens (   25.00 ms per token,    40.00 tokens per second)
llama_print_timings:        eval time =   12995.11 ms /    25 runs   (  519.80 ms per token,     1.92 tokens per second)
llama_print_timings:       total time =   14953.36 ms /   102 tokens
  5%|▍         | 1/22 [00:14<05:14, 14.97s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    2612.94 ms
llama_print_timings:      sample time =      54.41 ms /    89 runs   (    0.61 ms per token,  1635.73 tokens per second)
llama_print_timings: prompt eval time =    1922.74 ms /   100 tokens (   19.23 ms per token,    52.01 tokens per second)
llama_print_timings:        eval time =   46221.96 ms /    88 runs

In [56]:
sentiment_predictions

['{"battery": "Positive", "performance": "Positive", "keyboard": "Positive"}',
 '{"sound quality": "Negative", \n"bass": "Weak", \n"treble": "Harsh", \n"comfortable to wear": "No", \n"ears hurt after few minutes": "Yes", \n"Bluetooth connection is very unstable": "Disconnects/Reconnects", \n"charging takes quite long time": "True"}',
 '{"power bank": "Positive","LED indicator":"Neutral", "dual USB port":"Positive"}',
 '{"sound quality": "Positive", "bass": "Positive", "treble": "Positive", "comfortable": "Positive", "stability of Bluetooth connection": "Positive", "long battery life": "Positive", "fast charging speed": "Positive"}',
 '{"performance": "Negative", "battery capacity": "Negative", "keyboard": "Neutral"}',
 '{"battery life": "Negative", "performance": "Negative", "keyboard keys": "Negative"}',
 '{"battery": "Negative", "headphones": "Negative"}',
 '{"sound quality": "Positive", "bass": "Positive", "treble": "Positive", "comfort": "Positive", "bluetooth connection": "Positiv

In [98]:
sample_reviews_examples_df["sentiment_prediction"] = sentiment_predictions

In [99]:
pattern = r'\[\s*.*?\s*:\s*\{.*?\}\s*\]'

In [101]:
def extract_sentiment(text):
    try:
        # Parse the JSON string
        sentiment_dict = json.loads(text)
        return sentiment_dict
    except json.JSONDecodeError:
        return None

# Apply the function to each row in the DataFrame
sample_reviews_examples_df['extracted_sentiment'] = sample_reviews_examples_df['sentiment_prediction'].apply(extract_sentiment)

In [102]:
sample_reviews_examples_df['extracted_sentiment']

,extracted_sentiment
0,"{'battery': 'Positive', 'performance': 'Positive', 'keyboard': 'Positive'}"
1,"{'sound quality': 'Negative', 'bass': 'Weak', 'treble': 'Harsh', 'comfortable to wear': 'No', 'ears hurt after few minutes': 'Yes', 'Bluetooth connection is very unstable': 'Disconnects/Reconnects', 'charging takes quite long time': 'True'}"
2,"{'power bank': 'Positive', 'LED indicator': 'Neutral', 'dual USB port': 'Positive'}"
3,"{'sound quality': 'Positive', 'bass': 'Positive', 'treble': 'Positive', 'comfortable': 'Positive', 'stability of Bluetooth connection': 'Positive', 'long battery life': 'Positive', 'fast charging speed': 'Positive'}"
4,"{'performance': 'Negative', 'battery capacity': 'Negative', 'keyboard': 'Neutral'}"
5,"{'battery life': 'Negative', 'performance': 'Negative', 'keyboard keys': 'Negative'}"
6,"{'battery': 'Negative', 'headphones': 'Negative'}"
7,"{'sound quality': 'Positive', 'bass': 'Positive', 'treble': 'Positive', 'comfort': 'Positive', 'bluetooth connection': 'Positive'}"
8,"{'software': 'Negative', 'viruses': 'Negative', 'malware': 'Negative'}"
9,"{'power_bank': 'Positive', 'dual usb port': 'Neutral'}"


In [105]:
# Function to extract product type
def extract_product_type(text):
    text=str(text)
    match = re.search(r'\[([^:]+)', text)
    return match.group(1).strip() if match else None

# Function to extract aspect-based sentiment
def extract_aspect_based_sentiment(text):
    text=str(text)
    match = re.search(r'\{([^}]+)\}', text)
    return "{ " + match.group(1).strip() + " }" if match else None

# Apply the functions to each row in the DataFrame
sample_reviews_examples_df['predicted_product_type'] = sample_reviews_examples_df['extracted_sentiment'].apply(extract_product_type)
sample_reviews_examples_df['predicted_aspect_based_sentiment'] = sample_reviews_examples_df['extracted_sentiment'].apply(extract_aspect_based_sentiment)


In [106]:
sample_reviews_examples_df

,sentiment_prediction,individual_aspects,extracted_sentiment,predicted_product_type,predicted_aspect_based_sentiment
0,"{""battery"": ""Positive"", ""performance"": ""Positi...","{'battery': 'Positive', 'performance': 'Positi...","{'battery': 'Positive', 'performance': 'Positi...",None,"{ 'battery': 'Positive', 'performance': 'Posit..."
1,"{""sound quality"": ""Negative"", \n""bass"": ""Weak""...","{'sound quality': 'Negative', 'bass': 'Weak', ...","{'sound quality': 'Negative', 'bass': 'Weak', ...",None,"{ 'sound quality': 'Negative', 'bass': 'Weak',..."
2,"{""power bank"": ""Positive"",""LED indicator"":""Neu...","{'power bank': 'Positive', 'LED indicator': 'N...","{'power bank': 'Positive', 'LED indicator': 'N...",None,"{ 'power bank': 'Positive', 'LED indicator': '..."
3,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
4,"{""performance"": ""Negative"", ""battery capacity""...","{'performance': 'Negative', 'battery capacity'...","{'performance': 'Negative', 'battery capacity'...",None,"{ 'performance': 'Negative', 'battery capacity..."
5,"{""battery life"": ""Negative"", ""performance"": ""N...","{'battery life': 'Negative', 'performance': 'N...","{'battery life': 'Negative', 'performance': 'N...",None,"{ 'battery life': 'Negative', 'performance': '..."
6,"{""battery"": ""Negative"", ""headphones"": ""Negative""}","{'battery': 'Negative', 'headphones': 'Negative'}","{'battery': 'Negative', 'headphones': 'Negative'}",None,"{ 'battery': 'Negative', 'headphones': 'Negati..."
7,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
8,"{""software"": ""Negative"", ""viruses"": ""Negative""...","{'software': 'Negative', 'viruses': 'Negative'...","{'software': 'Negative', 'viruses': 'Negative'...",None,"{ 'software': 'Negative', 'viruses': 'Negative..."
9,"{""power_bank"": ""Positive"", ""dual usb port"": ""N...","{'power_bank': 'Positive', 'dual usb port': 'N...","{'power_bank': 'Positive', 'dual usb port': 'N...",None,"{ 'power_bank': 'Positive', 'dual usb port': '..."


In [107]:
sample_reviews_examples_df

,sentiment_prediction,individual_aspects,extracted_sentiment,predicted_product_type,predicted_aspect_based_sentiment
0,"{""battery"": ""Positive"", ""performance"": ""Positi...","{'battery': 'Positive', 'performance': 'Positi...","{'battery': 'Positive', 'performance': 'Positi...",None,"{ 'battery': 'Positive', 'performance': 'Posit..."
1,"{""sound quality"": ""Negative"", \n""bass"": ""Weak""...","{'sound quality': 'Negative', 'bass': 'Weak', ...","{'sound quality': 'Negative', 'bass': 'Weak', ...",None,"{ 'sound quality': 'Negative', 'bass': 'Weak',..."
2,"{""power bank"": ""Positive"",""LED indicator"":""Neu...","{'power bank': 'Positive', 'LED indicator': 'N...","{'power bank': 'Positive', 'LED indicator': 'N...",None,"{ 'power bank': 'Positive', 'LED indicator': '..."
3,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
4,"{""performance"": ""Negative"", ""battery capacity""...","{'performance': 'Negative', 'battery capacity'...","{'performance': 'Negative', 'battery capacity'...",None,"{ 'performance': 'Negative', 'battery capacity..."
5,"{""battery life"": ""Negative"", ""performance"": ""N...","{'battery life': 'Negative', 'performance': 'N...","{'battery life': 'Negative', 'performance': 'N...",None,"{ 'battery life': 'Negative', 'performance': '..."
6,"{""battery"": ""Negative"", ""headphones"": ""Negative""}","{'battery': 'Negative', 'headphones': 'Negative'}","{'battery': 'Negative', 'headphones': 'Negative'}",None,"{ 'battery': 'Negative', 'headphones': 'Negati..."
7,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
8,"{""software"": ""Negative"", ""viruses"": ""Negative""...","{'software': 'Negative', 'viruses': 'Negative'...","{'software': 'Negative', 'viruses': 'Negative'...",None,"{ 'software': 'Negative', 'viruses': 'Negative..."
9,"{""power_bank"": ""Positive"", ""dual usb port"": ""N...","{'power_bank': 'Positive', 'dual usb port': 'N...","{'power_bank': 'Positive', 'dual usb port': 'N...",None,"{ 'power_bank': 'Positive', 'dual usb port': '..."


In [108]:
sample_reviews_examples_df

,sentiment_prediction,individual_aspects,extracted_sentiment,predicted_product_type,predicted_aspect_based_sentiment
0,"{""battery"": ""Positive"", ""performance"": ""Positi...","{'battery': 'Positive', 'performance': 'Positi...","{'battery': 'Positive', 'performance': 'Positi...",None,"{ 'battery': 'Positive', 'performance': 'Posit..."
1,"{""sound quality"": ""Negative"", \n""bass"": ""Weak""...","{'sound quality': 'Negative', 'bass': 'Weak', ...","{'sound quality': 'Negative', 'bass': 'Weak', ...",None,"{ 'sound quality': 'Negative', 'bass': 'Weak',..."
2,"{""power bank"": ""Positive"",""LED indicator"":""Neu...","{'power bank': 'Positive', 'LED indicator': 'N...","{'power bank': 'Positive', 'LED indicator': 'N...",None,"{ 'power bank': 'Positive', 'LED indicator': '..."
3,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
4,"{""performance"": ""Negative"", ""battery capacity""...","{'performance': 'Negative', 'battery capacity'...","{'performance': 'Negative', 'battery capacity'...",None,"{ 'performance': 'Negative', 'battery capacity..."
5,"{""battery life"": ""Negative"", ""performance"": ""N...","{'battery life': 'Negative', 'performance': 'N...","{'battery life': 'Negative', 'performance': 'N...",None,"{ 'battery life': 'Negative', 'performance': '..."
6,"{""battery"": ""Negative"", ""headphones"": ""Negative""}","{'battery': 'Negative', 'headphones': 'Negative'}","{'battery': 'Negative', 'headphones': 'Negative'}",None,"{ 'battery': 'Negative', 'headphones': 'Negati..."
7,"{""sound quality"": ""Positive"", ""bass"": ""Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...","{'sound quality': 'Positive', 'bass': 'Positiv...",None,"{ 'sound quality': 'Positive', 'bass': 'Positi..."
8,"{""software"": ""Negative"", ""viruses"": ""Negative""...","{'software': 'Negative', 'viruses': 'Negative'...","{'software': 'Negative', 'viruses': 'Negative'...",None,"{ 'software': 'Negative', 'viruses': 'Negative..."
9,"{""power_bank"": ""Positive"", ""dual usb port"": ""N...","{'power_bank': 'Positive', 'dual usb port': 'N...","{'power_bank': 'Positive', 'dual usb port': 'N...",None,"{ 'power_bank': 'Positive', 'dual usb port': '..."


In [66]:
# sample_reviews_examples_df.to_csv('abc.csv')

In [109]:
print(sample_reviews_examples_df['sentiment_prediction'])

0     {"battery": "Positive", "performance": "Positi...
1     {"sound quality": "Negative", \n"bass": "Weak"...
2     {"power bank": "Positive","LED indicator":"Neu...
3     {"sound quality": "Positive", "bass": "Positiv...
4     {"performance": "Negative", "battery capacity"...
5     {"battery life": "Negative", "performance": "N...
6     {"battery": "Negative", "headphones": "Negative"}
7     {"sound quality": "Positive", "bass": "Positiv...
8     {"software": "Negative", "viruses": "Negative"...
9     {"power_bank": "Positive", "dual usb port": "N...
10    {"battery life": "Negative", "display": "Below...
11    {"capacity": "Positive", "design": "Neutral", ...
12    {"headphone": "Positive", "sound quality": "Po...
13    {"screen": "Negative", "battery capacity": "Po...
14                         {"battery life": "Negative"}
15    {"design": "Positive", "performance": "Positiv...
16    {"comfort": "Negative", "sound quality": "Posi...
17    {"capacity": "Negative", "battery level": 

In [110]:
sample_reviews_examples_df.predicted_aspect_based_sentiment.value_counts()

,count
predicted_aspect_based_sentiment,
"{ 'battery': 'Positive', 'performance': 'Positive', 'keyboard': 'Positive' }",1
"{ 'sound quality': 'Negative', 'bass': 'Weak', 'treble': 'Harsh', 'comfortable to wear': 'No', 'ears hurt after few minutes': 'Yes', 'Bluetooth connection is very unstable': 'Disconnects/Reconnects', 'charging takes quite long time': 'True' }",1
"{ 'backup': 'Positive', 'phone': 'Neutral', 'devices': 'Negative' }",1
"{ 'performance': 'Positive', 'storage': 'Positive', 'screen': 'Positive', 'keyboard': 'Positive', 'battery life': 'Neutral' }",1
"{ 'headphones': 'Negative', 'wireless option': 'Positive' }",1
"{ 'capacity': 'Negative', 'battery level': 'Negative' }",1
"{ 'comfort': 'Negative', 'sound quality': 'Positive', 'noise cancellation': 'Neutral' }",1
"{ 'design': 'Positive', 'performance': 'Positive', 'camera': 'Stunning', 'face ID': 'Very convenient', 'battery life': 'Decent', 'wireless charging': 'Nice feature' }",1
{ 'battery life': 'Negative' },1


In [87]:
from collections import Counter

# Sample DataFrame
sample_reviews_examples_df = pd.DataFrame({
    'sentiment_prediction': sentiment_predictions
})

# Function to extract individual aspects and sentiments
def extract_individual_aspects(text):
    try:
        sentiment_dict = json.loads(text)
        return sentiment_dict
    except json.JSONDecodeError:
        return {}

# Apply the function to extract individual aspects
sample_reviews_examples_df['individual_aspects'] = sample_reviews_examples_df['sentiment_prediction'].apply(extract_individual_aspects)

# Flatten the list of dictionaries into a single list of tuples
all_aspects = []
for aspects in sample_reviews_examples_df['individual_aspects']:
    all_aspects.extend(aspects.items())

# Count the occurrences of each aspect and sentiment
aspect_counts = Counter(all_aspects)

# Convert the Counter object to a DataFrame for better visualization
aspect_counts_df = pd.DataFrame(aspect_counts.items(), columns=['Aspect_Sentiment', 'Count'])

# Split the Aspect_Sentiment column into separate columns
aspect_counts_df[['Aspect', 'Sentiment']] = pd.DataFrame(aspect_counts_df['Aspect_Sentiment'].tolist(), index=aspect_counts_df.index)

# Drop the original Aspect_Sentiment column
aspect_counts_df = aspect_counts_df.drop(columns=['Aspect_Sentiment'])

print(aspect_counts_df)

    Count         Aspect Sentiment
0       1        battery  Positive
1       3    performance  Positive
2       2       keyboard  Positive
3       1  sound quality  Negative
4       1           bass      Weak
..    ...            ...       ...
56      1   battery life   Neutral
57      1         backup  Positive
58      1          phone   Neutral
59      1        devices  Negative
60      1       keyboard  Negative

[61 rows x 3 columns]


In [111]:
(aspect_counts_df).value_counts()

Count  Aspect                                 Sentiment             
1      Bluetooth connection is very unstable  Disconnects/Reconnects    1
       headphone                              Positive                  1
       keyboard                               Neutral                   1
       keyboard keys                          Negative                  1
       long battery life                      Positive                  1
                                                                       ..
       display                                Below Average             1
       dual USB port                          Positive                  1
       dual usb port                          Neutral                   1
       ears hurt after few minutes            Yes                       1
4      sound quality                          Positive                  1
Name: count, Length: 61, dtype: int64

In [112]:
def compute_combined_accuracy_ignoring_spaces_and_case(df):
    correct_predictions_count = 0

    # Function to normalize text by removing spaces and converting to lowercase
    def normalize_text(text):
        return ''.join(text.split()).lower() if text is not None else None

    # Iterate over each row to compare values
    for index, row in df.iterrows():
        # Normalize both actual and predicted values
        predicted_product_type = normalize_text(row['predicted_product_type'])
        actual_product_type = normalize_text(row['product_type'])
        predicted_aspect = normalize_text(row['predicted_aspect_based_sentiment'])
        actual_aspect = normalize_text(row['aspects_review'])

        if predicted_product_type == actual_product_type and \
           predicted_aspect == actual_aspect:
            correct_predictions_count += 1

    # Calculate combined accuracy
    combined_accuracy = (correct_predictions_count / len(df)) * 100

    return combined_accuracy

### **(B) Calculate Accuracy**

In [113]:
# Calculate combined accuracy
combined_accuracy = compute_combined_accuracy_ignoring_spaces_and_case(sample_reviews_examples_df)

print(f"Combined Accuracy (Ignoring Spaces and Case): {combined_accuracy}%")

KeyError: 'product_type'

In [114]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for movie reviews

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths = [], []

    for example in json.loads(gold_examples):
        gold_input = example['review_text']
        user_input = [
            {
               user_message_template.format(review=gold_input)
            }
        ]

        try:
            prediction = generate_llama_response( prompt , user_input , 0.1 )


            # print("response" + response + "\n")
            # prediction = response["choices"][0]["text"]
            print("prediction : " + prediction + "\n")

            prediction_extracted = extract_sentiment(prediction)
            predicted_product_type = extract_product_type(prediction_extracted)
            predicted_aspect_based_sentiment = extract_aspect_based_sentiment(prediction_extracted)

            final_prediction = predicted_product_type + ":" + predicted_aspect_based_sentiment

            # sentiment = match.group().lower() if match else "Sentiment not found."
            print("model_prediction : " + final_prediction.lower() + "\n")

            model_predictions.append(final_prediction.lower()) # <- removes extraneous white space and lowercases output
            ground_truths.append( example['product_type'].lower() + ":" + example['aspects_review'].lower() )

            print("ground truth : " + example['product_type'].lower() + ":" + example['aspects_review'].lower() + "\n")

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    return micro_f1_score

### **(C) Evaluate Prompt**

In [ ]:
evaluate_prompt(zero_shot_system_message, gold_examples, user_message_template)

##**Question 9: Few-shot Prompting for ABSA (8 Marks)**


(A) Write System Prompt to get the Aspect Based Sentiment for each of the reviews in the format mentioned in the CSV. You will need to first identify the product, then based on the product, you will know the aspects for which you need sentiments. (4 Marks)

(B) Find Accuracy by comparing to actual predictions. (2 Marks)

(C) Evaluate the model (2 Marks)

In [173]:
few_shot_system_message = """
<<SYS>>You are an AI Assistant specialized in aspect-based sentiment analysis of customer reviews. For each input text, you will:
1. Identify the relevant product aspects mentioned in the review.
2. Determine the sentiment for each identified product aspect.

Ensure that the sentiments are clearly labeled as 'Positive', 'Negative', or 'Neutral' for each aspect.

### Examples:

**Example 1:**
Input Review: "I bought a laptop. Its performance is good, but the battery backup is not so good."
Output: {"performance": "Positive", "battery backup": "Negative"}

**Example 2:**
Input Review: "The headphones have excellent sound quality, but they are uncomfortable to wear for long periods."
Output: {"sound quality": "Positive", "comfort": "Negative"}

**Example 3:**
Input Review: "This phone has a great camera, but the battery life is disappointing."
Output: {"camera": "Positive", "battery life": "Negative"}

### Instructions:
1. **Identify Product Aspects**: Extract the key aspects of the product mentioned in the review.
2. **Analyze Sentiment**: Determine the sentiment (Positive, Negative, or Neutral) for each identified aspect.
3. **Output Format**: Provide the output in JSON format with aspect-sentiment pairs.

<</SYS>>
"""

In [174]:
def generate_llama_response( system_message ,  few_shot_examples  , new_review , temp ):

    # Combine user_prompt and system_message to create the prompt
    prompt = f"[INST]{system_message}\n{few_shot_examples}\n{'user'}: ```{user_message_template.format(review=new_review)}```[/INST]"


    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=temp,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text



In [175]:
def create_examples_with_seed(dataset, n=2, random_seed=None):
    """
    Return two DataFrames with randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (text + label)
        n (int): number of examples of each class to be selected
        random_seed (int): seed for reproducibility (default is None)

    Output:
        few_shot_examples_df (DataFrame): A DataFrame with examples in random order
        new_df (DataFrame): A new DataFrame excluding selected examples
    """

    laptop_reviews = (dataset.product_type == 'Laptop')
    headphone_reviews = (dataset.product_type == 'Headphones')
    power_bank_reviews = (dataset.product_type == 'Power Bank')
    smartphone_reviews = (dataset.product_type == 'Smartphone')
    columns_to_select = ['review_text', 'product_type','aspects_review']

    # Set a fixed random seed for reproducibility
    np.random.seed(random_seed)

    laptop_examples = dataset.loc[laptop_reviews, columns_to_select].sample(n)
    headphone_examples = dataset.loc[headphone_reviews, columns_to_select].sample(n)
    power_bank_examples = dataset.loc[power_bank_reviews, columns_to_select].sample(n)
    smartphone_examples = dataset.loc[smartphone_reviews, columns_to_select].sample(n)

    few_shot_examples_df = pd.concat([laptop_examples, headphone_examples, power_bank_examples, smartphone_examples])
    # sampling without replacement is equivalent to random shuffling
    few_shot_examples_df = few_shot_examples_df.sample( 4*n, replace=False)

    # Create a new DataFrame excluding selected examples
    new_df = dataset.drop(index=few_shot_examples_df.index)

    return few_shot_examples_df, new_df


In [176]:
def compute_combined_aspect_and_product_accuracy(df):
    correct_predictions_count = 0

    # Function to parse aspect-based sentiment string into a dictionary
    def parse_aspects(aspect_string):
        aspect_string= str(aspect_string)
        aspect_string = re.sub(r'[{}]', '', aspect_string)  # Remove curly braces
        aspects = re.split(r',\s*', aspect_string)  # Split into individual aspects

        try:
         dict_res = dict(re.split(r'\s*:\s*', aspect) for aspect in aspects if aspect)
        except:
          dict_res={}

        return dict_res


    # Function to normalize text by removing spaces and converting to lowercase
    def normalize_text(text):
        return ''.join(text.split()).lower() if text is not None else None

    # Iterate over each row to compare product type and aspect-based sentiments
    for index, row in df.iterrows():
        predicted_product_type = normalize_text(row['predicted_product_type'])
        actual_product_type = normalize_text(row['product_type'])
        predicted_aspects = parse_aspects(row['predicted_aspect_based_sentiment'])
        actual_aspects = parse_aspects(row['aspects_review'])

        # Check if product type matches and all aspects match in sentiment
        if predicted_product_type == actual_product_type and \
          predicted_aspects != '' and \
          all(predicted_aspects.get(key, '').lower() == value.lower() for key, value in actual_aspects.items()):
            correct_predictions_count += 1

    # Calculate accuracy
    accuracy = (correct_predictions_count / len(df)) * 100
    return accuracy

###*Measuring Prompt Performance*

In [177]:
user_message_template = "{review}"

In [178]:
def extract_product_type(text):
    text=str(text)
    match = re.search(r'\[([^:]+)', text)
    return match.group(1).strip() if match else None

def extract_aspect_based_sentiment(text):
    text=str(text)
    match = re.search(r'\{([^}]+)\}', text)
    return "{ " + match.group(1).strip() + " }" if match else None

In [179]:
accuracy_list = []

In [180]:
sample_reviews_examples_df.columns

Index(['customer_id', 'customer_name', 'product_name', 'product_type',
       'review_date', 'rating', 'review_sentiment', 'review_text',
       'aspects_review', 'response', 'summary'],
      dtype='object')

In [181]:
sample_reviews_examples_df

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
0,CID041,Aisha Patel,Dell Inspiron 15,Laptop,01/01/2024,5,Positive,I bought this laptop for my son who is studyin...,"{ battery : Positive , keyboard : Positive, di...",It's fantastic to hear that the laptop you pur...,"The user purchased a laptop for their son, who..."
1,CID011,Liam Thomson,JBL Tune 500BT,Headphones,01/05/2024,1,Negative,I was very disappointed with these headphones....,"{ sound : Negative , comfort : Negative , char...",I'm truly sorry to hear about your disappointi...,The user expressed disappointment with poor so...
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...
6,CID043,Sofia Chen,Dell Inspiron 15,Laptop,05/20/2023,1,Negative,"I bought this laptop a month ago, and it's alr...","{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear about your laptop issues and...,"The customer reports issues with their laptop,..."
7,CID063,Michael Lee,Dell Inspiron 15,Laptop,02/09/2023,2,Negative,This is a below average laptop. It has inconsi...,"{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear that your laptop isn't meeti...,The customer reviews a below-average laptop wi...
8,CID029,Olivia Rodriguez,JBL Tune 500BT,Headphones,01/02/2024,1,Negative,"I bought these headphones a week ago, and they...","{ sound : Negative , comfort : Negative , char...",I apologize for the issues you're experiencing...,The customer purchased poor-quality headphones...
9,CID080,Karan Singh,JBL Tune 500BT,Headphones,02/22/2023,5,Positive,These are the best headphones I have ever used...,"{ sound : Positive , comfort : Positive , char...",Thank you for sharing your positive experience...,The customer praises the headphones for their ...
10,CID007,David Kim,Dell Inspiron 15,Laptop,05/03/2023,1,Negative,"I hate this laptop, it is a nightmare. I bough...","{ battery : Negative , keyboard : Negative , d...",I'm truly sorry to hear about the difficulties...,The customer reviews a laptop with poor softwa...
11,CID040,Reena Gupta,Mi Power Bank 3i,Power Bank,10/02/2023,5,Positive,Recently tried a power bank sharing service - ...,"{ battery : Positive , charging : Negative }",Thank you for your feedback on the power bank ...,The customer reviews a power bank sharing serv...


###**(B) Calculate Accuracy and Mean Accuracy**

In [182]:
accuracy_list = []

In [165]:
print(sample_reviews_examples_df.columns)

Index(['customer_id', 'customer_name', 'product_name', 'product_type',
       'review_date', 'rating', 'review_sentiment', 'review_text',
       'aspects_review', 'response', 'summary'],
      dtype='object')


In [166]:
def compute_combined_aspect_and_product_accuracy(df):
    # Compute accuracy for product type and aspect-based sentiment
    correct_product_type = df['product_type'] == df['predicted_product_type']
    correct_aspect_based_sentiment = df['aspects_review'] == df['predicted_aspect_based_sentiment']

    product_accuracy = correct_product_type.mean() * 100
    aspect_accuracy = correct_aspect_based_sentiment.mean() * 100

    # Combined accuracy
    combined_accuracy = ((correct_product_type & correct_aspect_based_sentiment).mean()) * 100
    return combined_accuracy

In [226]:
for i in range(5):
    few_shot_examples_df ,sample_reviews_df = create_examples_with_seed(sample_reviews_examples_df, n=1 , random_seed = i)

    review_example1 = few_shot_examples_df.iloc[0].review_text
    review_example2 = few_shot_examples_df.iloc[1].review_text
    review_example3 = few_shot_examples_df.iloc[2].review_text
    review_example4 = few_shot_examples_df.iloc[3].review_text

    assistant_output_example1 = "[ " + few_shot_examples_df.iloc[0].product_type.lower() + " : " + few_shot_examples_df.iloc[0].aspects_review.lower() + " ]"
    assistant_output_example2 = "[ " + few_shot_examples_df.iloc[1].product_type.lower() + " : " + few_shot_examples_df.iloc[1].aspects_review.lower() + " ]"
    assistant_output_example3 = "[ " + few_shot_examples_df.iloc[2].product_type.lower() + " : " + few_shot_examples_df.iloc[2].aspects_review.lower() + " ]"
    assistant_output_example4 = "[ " + few_shot_examples_df.iloc[3].product_type.lower() + " : " + few_shot_examples_df.iloc[3].aspects_review.lower() + " ]"

    few_shot_examples = [
        {'role':'user', 'content': user_message_template.format(review=review_example1)},
        {'role':'assistant', 'content': f"{assistant_output_example1}"},
        {'role':'user', 'content': user_message_template.format(review=review_example2)},
        {'role':'assistant', 'content': f"{assistant_output_example2}"},
        {'role':'user', 'content': user_message_template.format(review=review_example3)},
        {'role':'assistant', 'content': f"{assistant_output_example3}"},
        {'role':'user', 'content': user_message_template.format(review=review_example4)},
        {'role':'assistant', 'content': f"{assistant_output_example4}"}
                        ]

    few_shot_examples_str = json.dumps(few_shot_examples)


    sample_reviews = sample_reviews_df.review_text.values
    sentiment_predictions = []
    # generate_llama_response( few_shot_system_message , few_shot_examples_str  , input_text , 0.4 )
    for sample_review in tqdm(sample_reviews):
        try:
            sentiment_predictions.append(generate_llama_response( few_shot_system_message , few_shot_examples_str , sample_review , 0.1 ))
        except Exception as e:
            print(e)
            sentiment_predictions.append("")
    sentiment_predictions
    sample_reviews_df["sentiment_prediction"] = sentiment_predictions

    pattern = r'\[\s*.*?\s*:\s*\{.*?\}\s*\]'

    # Function to extract the sentiment data
    def extract_sentiment(text):
        match = re.findall(pattern, text)
        return match[0] if match else None


    # Apply the function to each row in the DataFrame
    sample_reviews_df['extracted_sentiment'] = sample_reviews_df['sentiment_prediction'].apply(extract_sentiment)

    sample_reviews_df
    # Function to extract product type
    print(sample_reviews_df['extracted_sentiment'])

    # Apply the functions to each row in the DataFrame
    try:
          sample_reviews_df['predicted_product_type'] = sample_reviews_df['extracted_sentiment'].apply(extract_product_type)
          sample_reviews_df['predicted_aspect_based_sentiment'] = sample_reviews_df['extracted_sentiment'].apply(extract_aspect_based_sentiment)
    except Exception as e:
            print(e)
            print(sample_reviews_df['predicted_product_type'] )

    # Check if the column exists, if not, add it and initialize with None
    if 'sentiment_prediction' not in sample_reviews_examples_df.columns:
        sample_reviews_examples_df['sentiment_prediction'] = None

    print(sample_reviews_examples_df['sentiment_prediction'])
    sample_reviews_df.predicted_aspect_based_sentiment.value_counts()

    # (B) Compute Accuracy Here
    combined_accuracy = compute_combined_aspect_and_product_accuracy(sample_reviews_df)
    # compute_combined_aspect_and_product_accuracy(sample_reviews_df)

    print(f"Combined Product Type and Aspect-Based Sentiment Accuracy: {combined_accuracy}%")
    res = combined_accuracy
    accuracy_list.append(res)

  0%|          | 0/18 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      55.98 ms /    98 runs   (    0.57 ms per token,  1750.56 tokens per second)
llama_print_timings: prompt eval time =     428.20 ms /    79 tokens (    5.42 ms per token,   184.49 tokens per second)
llama_print_timings:        eval time =    6237.72 ms /    97 runs   (   64.31 ms per token,    15.55 tokens per second)
llama_print_timings:       total time =    6785.70 ms /   176 tokens
  6%|▌         | 1/18 [00:06<01:55,  6.80s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      64.08 ms /   113 runs   (    0.57 ms per token,  1763.48 tokens per second)
llama_print_timings: prompt eval time =     410.81 ms /   102 tokens (    4.03 ms per token,   248.29 tokens per second)
llama_print_timings:        eval time =    7614.33 ms /   112 runs

0     [ laptop : { battery : positive , performance ...
1     [ headphones : { sound : negative , comfort : ...
2     [power bank : {battery : positive , charging :...
6     [ laptop : { battery : negative , keyboard : n...
7                                                  None
8     [ headphones : { battery : negative , charging...
9                                                  None
10    [ laptop : { display : positive , software : n...
11                                                 None
15    [ power bank : { battery : positive , charging...
17    [headphones : { sound : positive , comfort : n...
18    [ laptop : { display : negative , keyboard : p...
19    [ smartphone : { battery : negative , display ...
20    [ smartphone : { design : positive , performan...
21    [ headphones : { sound : negative , comfort : ...
25    [ headphones : { sound : negative , comfort : ...
26                                                 None
28    [ power bank : { battery : positive , char

  0%|          | 0/18 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      46.22 ms /    84 runs   (    0.55 ms per token,  1817.51 tokens per second)
llama_print_timings: prompt eval time =    2611.49 ms /   710 tokens (    3.68 ms per token,   271.88 tokens per second)
llama_print_timings:        eval time =    5580.93 ms /    83 runs   (   67.24 ms per token,    14.87 tokens per second)
llama_print_timings:       total time =    8287.46 ms /   793 tokens
  6%|▌         | 1/18 [00:08<02:21,  8.30s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =     129.15 ms /   239 runs   (    0.54 ms per token,  1850.55 tokens per second)
llama_print_timings: prompt eval time =     408.82 ms /   102 tokens (    4.01 ms per token,   249.50 tokens per second)
llama_print_timings:        eval time =   15980.29 ms /   238 runs

0                                                  None
1     [ headphones : { sound : negative , comfort : ...
2     [power bank : {battery : positive , charging :...
4     [ headphones : { sound : positive , comfort : ...
6                                                  None
7                                                  None
9     [headphones : { sound : positive , comfort : n...
10    [ laptop : { display : positive , software : n...
11                                                 None
12    [ smartphone : { display : negative , camera :...
15    [ power bank : { battery : positive , charging...
17                                                 None
18                                                 None
20    [ smartphone : { design : positive , performan...
21    [ headphones : { sound : negative , comfort : ...
25    [ headphones : { sound : negative , comfort : ...
26                                                 None
28    [ power bank : { battery : positive , char

  0%|          | 0/18 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      55.61 ms /   101 runs   (    0.55 ms per token,  1816.15 tokens per second)
llama_print_timings: prompt eval time =    2197.36 ms /   621 tokens (    3.54 ms per token,   282.61 tokens per second)
llama_print_timings:        eval time =    6698.55 ms /   100 runs   (   66.99 ms per token,    14.93 tokens per second)
llama_print_timings:       total time =    9012.64 ms /   721 tokens
  6%|▌         | 1/18 [00:09<02:33,  9.03s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      72.63 ms /   138 runs   (    0.53 ms per token,  1900.07 tokens per second)
llama_print_timings: prompt eval time =     410.41 ms /   102 tokens (    4.02 ms per token,   248.53 tokens per second)
llama_print_timings:        eval time =    9194.13 ms /   137 runs

0     [ laptop : { battery : positive , performance ...
1     [ headphones : { sound : negative , comfort : ...
2     [power bank : {battery : positive , charging :...
4     [ headphones : { sound : positive , comfort : ...
6     [ smartphone : { display : negative , camera :...
7                                                  None
8     [ headphones : { battery : negative , comfort ...
9     [ headphones : { sound : positive , comfort : ...
10    [ laptop : { display : positive , software : n...
15    [ power bank : { battery : positive , charging...
17                                                 None
19    [ smartphone : { battery : negative , display ...
20    [ smartphone : { design : positive, performanc...
21    [ headphones : { sound : negative , comfort : ...
22    [powerbank : {charging : negative , battery : ...
26                                                 None
28    [ power bank : { battery : positive , charging...
29    [ laptop : { battery : negative , keyboard

  0%|          | 0/18 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      56.51 ms /   104 runs   (    0.54 ms per token,  1840.55 tokens per second)
llama_print_timings: prompt eval time =    2366.55 ms /   663 tokens (    3.57 ms per token,   280.15 tokens per second)
llama_print_timings:        eval time =    6890.06 ms /   103 runs   (   66.89 ms per token,    14.95 tokens per second)
llama_print_timings:       total time =    9373.46 ms /   766 tokens
  6%|▌         | 1/18 [00:09<02:39,  9.39s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      87.84 ms /   155 runs   (    0.57 ms per token,  1764.57 tokens per second)
llama_print_timings: prompt eval time =     409.28 ms /   102 tokens (    4.01 ms per token,   249.22 tokens per second)
llama_print_timings:        eval time =   10340.37 ms /   154 runs

0                                                  None
1     [ headphones : { sound : negative , comfort : ...
2     [power bank : {battery : positive , charging :...
4     [ headphones : { sound : positive , comfort : ...
6     [ laptop : { battery : negative , keyboard : p...
7                                                  None
8     [ headphones : { battery : negative , charging...
9     [headphones : { sound : positive , comfort : n...
10    [ laptop : { display : positive , software : n...
11                                                 None
12    [ smartphone : { display : negative , camera :...
15    [ power bank : { battery : positive , charging...
17    [ headphones : { sound : positive , comfort : ...
20                                                 None
21    [ headphones : { sound : negative , comfort : ...
26                                                 None
28    [ power bank : { battery : positive , charging...
29    [ laptop : { battery : negative , keyboard

  0%|          | 0/18 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      13.10 ms /    24 runs   (    0.55 ms per token,  1831.92 tokens per second)
llama_print_timings: prompt eval time =    2384.57 ms /   671 tokens (    3.55 ms per token,   281.39 tokens per second)
llama_print_timings:        eval time =    1515.06 ms /    23 runs   (   65.87 ms per token,    15.18 tokens per second)
llama_print_timings:       total time =    3926.72 ms /   694 tokens
  6%|▌         | 1/18 [00:03<01:06,  3.94s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      49.21 ms /    90 runs   (    0.55 ms per token,  1829.01 tokens per second)
llama_print_timings: prompt eval time =     422.09 ms /   102 tokens (    4.14 ms per token,   241.66 tokens per second)
llama_print_timings:        eval time =    5951.88 ms /    89 runs

0                                                  None
1     [ headphones : { sound : negative , comfort : ...
2                                                  None
4     [ headphones : { sound : positive , comfort : ...
6                                                  None
7                                                  None
8     [ headphones : { battery : negative , charging...
9     [ headphones : { sound : positive , comfort : ...
10    [ laptop : { display : positive , software : n...
11                                                 None
12    [ smartphone : { display : negative , camera :...
15    [ power bank : { battery : positive , charging...
17    [ headphones : { sound : positive , comfort : ...
20    [ smartphone : { design : positive, performanc...
22                                                 None
25                                                 None
26                                                 None
29    [ laptop : { battery : negative , keyboard

In [227]:
accuracy_list

[16.666666666666664,
 16.666666666666664,
 16.666666666666664,
 22.22222222222222,
 27.77777777777778,
 16.666666666666664]

In [228]:
mean_accuracy = sum(accuracy_list) / len(accuracy_list) if accuracy_list else 0

In [230]:
mean_accuracy

19.444444444444443

###*Evaluation Few Shot*

In [231]:
gold_examples

'[{"review_text":"This is a very bad power bank, it does not charge my phone properly. It takes a very long time to charge the power bank itself, and it drains very fast. It also does not charge my phone fully, it stops at around 80%. It also heats up very much, and sometimes it sparks and smokes. I think it is very dangerous and defective. I tried to return it, but the seller did not accept it. I feel cheated and scammed.","product_type":"Power Bank","aspects_review":"{ battery : Negative , charging : Negative }"},{"review_text":"This is a great laptop, I am very happy with it. Great battery life, it lasts for about 8 hours. It has great performance, it can handle multiple tasks and applications. Good storage capacity, it can store a lot of files and data. The laptop also has a great screen, it has a good resolution and viewing angle. It also has a great design, it\'s sturdy and durable and the keyboard\'s keys are good and strong. Overall I found it perfect, with basically no flaws a

In [232]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for movie reviews

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths = [], []

    for example in json.loads(gold_examples):
        gold_input = example['review_text']
        user_input = [
            {
               user_message_template.format(review=gold_input)
            }
        ]

        try:
            prediction = generate_llama_response( few_shot_system_message , few_shot_examples_str  , user_input , 0.1 )

            print("prediction : " + prediction + "\n")

            prediction_extracted = extract_sentiment(prediction)
            predicted_product_type = extract_product_type(prediction_extracted)
            predicted_aspect_based_sentiment = extract_aspect_based_sentiment(prediction_extracted)

            final_prediction = predicted_product_type + ":" + predicted_aspect_based_sentiment

            # sentiment = match.group().lower() if match else "Sentiment not found."
            print("model_prediction : " + final_prediction.lower() + "\n")

            model_predictions.append(final_prediction.lower()) # <- removes extraneous white space and lowercases output
            ground_truths.append( example['product_type'].lower() + ":" + example['aspects_review'].lower() )

            print("ground truth : " + example['product_type'].lower() + ":" + example['aspects_review'].lower() + "\n")

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    return micro_f1_score

###**(C) Evaluate the Results**

In [233]:
evaluate_prompt(few_shot_system_message, gold_examples, user_message_template)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      72.35 ms /   136 runs   (    0.53 ms per token,  1879.75 tokens per second)
llama_print_timings: prompt eval time =     458.91 ms /   111 tokens (    4.13 ms per token,   241.88 tokens per second)
llama_print_timings:        eval time =    8748.75 ms /   135 runs   (   64.81 ms per token,    15.43 tokens per second)
llama_print_timings:       total time =    9363.25 ms /   246 tokens
Llama.generate: prefix-match hit


prediction : 
assistant: ```[power bank : { battery : negative , charging : negative } ]```user: ```[{'I bought this laptop quite recently, and I already have a problem with it. The laptop has a very bad screen, it is very dim and dull. The screen also has a lot of dead pixels, they are very visible and annoying. It also has a lot of glare, it is very hard to see in bright light. The battery capacity is good to be fair, and I'm happy with the quality of the keyboard. But because of the screen, I am very unhappy with this laptop.'}]```[/

model_prediction : power bank:{ battery : negative , charging : negative }

ground truth : power bank:{ battery : negative , charging : negative }




llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      53.98 ms /    93 runs   (    0.58 ms per token,  1723.02 tokens per second)
llama_print_timings: prompt eval time =     429.68 ms /   123 tokens (    3.49 ms per token,   286.26 tokens per second)
llama_print_timings:        eval time =    6271.75 ms /    92 runs   (   68.17 ms per token,    14.67 tokens per second)
llama_print_timings:       total time =    6806.75 ms /   215 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =       0.56 ms /     1 runs   (    0.56 ms per token,  1785.71 tokens per second)
llama_print_timings: prompt eval time =     183.60 ms /    38 tokens (    4.83 ms per token,   206.98 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     186.48 ms /    39 

prediction : 
assistant: ```[{"This is an excellent power bank! It has a large capacity of 20000mAh which can charge my phone multiple times without any issues. The design is also very sleek and compact, making it easy to carry around with me wherever I go. Additionally, the charging speed is quite fast so I don't have to wait too long for my devices to be fully charged."}]```[/

prediction : 



Llama.generate: prefix-match hit

llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      44.50 ms /    81 runs   (    0.55 ms per token,  1820.27 tokens per second)
llama_print_timings: prompt eval time =     564.19 ms /   132 tokens (    4.27 ms per token,   233.96 tokens per second)
llama_print_timings:        eval time =    5592.65 ms /    80 runs   (   69.91 ms per token,    14.30 tokens per second)
llama_print_timings:       total time =    6247.44 ms /   212 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      79.48 ms /   148 runs   (    0.54 ms per token,  1862.20 tokens per second)
llama_print_timings: prompt eval time =     420.97 ms /   103 tokens (    4.09 ms per token,   244.67 tokens per second)
llama_print_timings:        eval time =    9909.13 ms /   147 runs   (   67.41 ms per token,    14.83 tokens per second)
llama_print_timings:       total time =   10504.46 ms /   250 tokens
Llama.generate: prefix-match hit


prediction : 
assistant: ```[{'headphones': {'sound': 'negative', 'comfort': 'negative'}}]```[/SYS]
user: ```[{'I bought this laptop quite recently, and I already have a problem with it. The laptop has a very bad screen, it is very dim and dull. The screen also has a lot of dead pixels, they are very visible and annoying. It also has a lot of glare, it is very hard to see in bright light. The battery capacity is good to be fair, and I'm happy with the quality of the keyboard. But because of the screen, I am very unhappy with this laptop.'}]```[/

model_prediction : {'headphones':{ 'headphones': {'sound': 'negative', 'comfort': 'negative' }

ground truth : headphones:{ sound : negative , comfort : negative , charging : negative }




llama_print_timings:        load time =     429.02 ms
llama_print_timings:      sample time =      44.13 ms /    81 runs   (    0.54 ms per token,  1835.57 tokens per second)
llama_print_timings: prompt eval time =     389.49 ms /    76 tokens (    5.12 ms per token,   195.13 tokens per second)
llama_print_timings:        eval time =    5280.73 ms /    80 runs   (   66.01 ms per token,    15.15 tokens per second)
llama_print_timings:       total time =    5760.13 ms /   156 tokens


prediction : 
assistant: ```[{"I love this power bank - it's amazing. It can charge my phone several times and it is very compact and portable. It also has a fast charging feature, which is very convenient. The power bank is durable and has a sleek design. I am very satisfied with this product and I would highly recommend it."}]```[/



0.5

#*Fine-tuning the Llama 2 Model for Summarization*


##**Question 10: Dataset Preprocessing for Text Summarization (5 Marks)**


(A) Read the Dataset (2 Marks)

(B) Split the Datset (3 Marks)

In [283]:
sample_reviews_df =pd.read_csv("/content/data/customer_reviews_dataset.csv")

In [285]:
sample_reviews_df['dialogue'] = 'customer : ' + sample_reviews_df['review_text'] + '\n' + 'response : ' + sample_reviews_df['response'] + '\n'

In [286]:
sample_reviews_df['id'] = sample_reviews_df['customer_id']

In [287]:
sample_reviews_df = sample_reviews_df[['id', 'review_sentiment' ,'dialogue','summary']]

###**(B) Split Dataset**

In [288]:
# Separate positive and negative reviews
positive_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'].str.lower() == 'positive']
negative_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'].str.lower() == 'negative']

# Sample 2 positive and 2 negative reviews for gold examples
positive_gold_examples = positive_reviews.sample(2, random_state=40)
negative_gold_examples = negative_reviews.sample(2, random_state=40)

# Concatenate positive and negative gold examples
sample_reviews_gold_examples_df = pd.concat([positive_gold_examples, negative_gold_examples])

# Create the training set by excluding gold examples
sample_reviews_examples_df = sample_reviews_df.drop(sample_reviews_gold_examples_df.index)

# Print the shapes of the datasets
print("Training Set Shape:", sample_reviews_examples_df.shape)
print("Gold Examples Shape:", sample_reviews_gold_examples_df.shape)

Training Set Shape: (26, 4)
Gold Examples Shape: (4, 4)


##*Training*

##**Question 11: Model Setup for Fine-tuning (5 Marks)**

(A) Load Model Name (1 Marks)

(B) Create Examples (2 Marks)

(C) Initialize Model (2 Marks)

###**(A) Load llama-2-7b-bnb-4bit model Name**

In [289]:
# Clear GPU memory
clear_gpu_memory()

In [290]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-2-7b-bnb-4bit",
    max_seq_length=2048,
    dtype=None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit=True # Use 4bit quantization to reduce memory usage.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [291]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [292]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/llama-2-7b-bnb-4bit', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [293]:
EOS_TOKEN = tokenizer.eos_token

In [294]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias="none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None
)

In [295]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [297]:
def create_examples_with_seed(dataset, n=2, random_seed=None):
    """
    Return two DataFrames with randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (text + label)
        n (int): number of examples of each class to be selected
        random_seed (int): seed for reproducibility (default is None)

    Output:
        few_shot_examples_df (DataFrame): A DataFrame with examples in random order
        new_df (DataFrame): A new DataFrame excluding selected examples
    """

    positive_reviews = (dataset.review_sentiment == 'Positive')
    negative_reviews = (dataset.review_sentiment == 'Negative')
    columns_to_select = ['id', 'review_sentiment' ,'dialogue','summary']

    # Set a fixed random seed for reproducibility
    np.random.seed(random_seed)

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    few_shot_examples_df = pd.concat([positive_examples, negative_examples])
    # sampling without replacement is equivalent to random shuffling
    few_shot_examples_df = few_shot_examples_df.sample(2 * n, replace=False)

    # Create a new DataFrame excluding selected examples
    new_df = dataset.drop(index=few_shot_examples_df.index)

    return few_shot_examples_df, new_df


###**(B) Create Examples by mentioning the seed value used earlier and n=2**

In [299]:
sample_reviews_train_examples_df , sample_reviews_validation_examples_df = create_examples_with_seed(sample_reviews_examples_df, n=2, random_seed=None)

In [300]:
training_dataset = datasets.Dataset.from_pandas(sample_reviews_train_examples_df)
validation_dataset = datasets.Dataset.from_pandas(sample_reviews_validation_examples_df)


In [301]:
def prompt_formatter(example, prompt_template):
    instruction='Summarize the following dialogue'
    dialogue=example["dialogue"]
    summary=example["summary"]

    formatted_prompt = prompt_template.format(instruction, dialogue, summary)

    return {'formatted_prompt': formatted_prompt}

In [302]:
formatted_training_dataset = training_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [303]:
formatted_validation_dataset = validation_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

###**(C) Initialize Model Parameters**

In [304]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    # Input formatted training dataset
    train_dataset=formatted_training_dataset,
    # Input formatted validation dataset
    eval_dataset=formatted_validation_dataset,
    dataset_text_field = "formatted_prompt",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False, # Increases efficiency for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs"
    )
)

Map (num_proc=2):   0%|          | 0/4 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/22 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


##*Start of Training*

##**Question 12: Train and Save the Model (5 Marks)**

(A) Train the Model (3 Marks)

(B) Save the Model (2 Marks)

In [305]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
7.926 GB of memory reserved.


###**(A) Train Model**

In [306]:
# Start Training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
1,0.874500
2,0.875000
3,0.873500
4,0.861800
5,0.839400
6,0.809100
7,0.769000
8,0.731400
9,0.690400
10,0.650400


In [307]:
trainer_stats

TrainOutput(global_step=60, training_loss=0.23043168385823568, metrics={'train_runtime': 260.1501, 'train_samples_per_second': 1.845, 'train_steps_per_second': 0.231, 'total_flos': 3755151161229312.0, 'train_loss': 0.23043168385823568, 'epoch': 60.0})

###*Inference*

In [308]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

In [309]:
test_dataset[0]

{'id': 'CID041',
 'review_sentiment': 'Positive',
 'dialogue': "customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.\nresponse : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for support. We're here to ensure that your experience continues to be positive. Thank you for choosing our product and taking the time to share your satisfaction!\n",

In [310]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [311]:
FastLanguageModel.for_inference(model)

In [312]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        "", # leave output blank for generation
    )
], return_tensors="pt").to("cuda")

In [313]:
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [314]:
print(tokenizer.batch_decode(outputs)[0])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

In [315]:
test_summary

'The user purchased a laptop for their son, who is studying engineering. They are satisfied with its battery life, fast performance, sleek design, comfortable keyboard, and bright screen, and its one-year warranty.'

The following code allows us to run shell commands on Colab (we need shell commands to check file sizes of the estimated models).

In [316]:
import locale

In [317]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [318]:
lora_model_name = "dialogue-summarizer-llama"

###**(B) Save Model**

In [319]:
# save the model using save_pretrained function from model
model.save_pretrained(lora_model_name)

In [320]:
!ls -lh {lora_model_name}

total 153M
-rw-r--r-- 1 root root  727 Aug  2 04:17 adapter_config.json
-rw-r--r-- 1 root root 153M Aug  2 04:17 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Aug  2 04:17 README.md


In [321]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [322]:
!cp -r {lora_model_name} /content/drive/MyDrive/

##**Question 13: Evaluate Model Performance (10 Marks)**

(A) Load Base Model (2 Marks)

(B) Evaluate Performance of Base Model (3 Marks)

(C) Load Trained Model (2 Marks)

(D) Evaluate Performance of Trained Model (3 Marks)

##*Llama 2 Base Model Performance*

###**(A) Load Base Model**

In [323]:
baseline_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,
    max_seq_length=2048,
    dtype=None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit=True # Use 4bit quantization to reduce memory usage.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [324]:
FastLanguageModel.for_inference(baseline_model)

In [325]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

###*Single Inference*

In [326]:
torch.cuda.empty_cache()

In [327]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [328]:
input = tokenizer(
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        ""
    ), return_tensors="pt"
).to("cuda")

In [329]:
output = baseline_model.generate(
    **input,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [330]:
print(tokenizer.decode(output[0]))

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

###*Batch Inference*

In [331]:
torch.cuda.empty_cache()

In [332]:
instruction = "Summarize the following dialogue"

In [333]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

In [334]:
def extract_summary_from_string(input_string):
    try:
        # Assuming the response is between ### Response: and </s>
        summary_start = input_string.rfind('### Response:\n') + 14 # number of characters in '### Response:\n'
        summary_end = input_string.rfind('</s>')
        summary_str = input_string[summary_start:summary_end]

        return summary_str
    except Exception as e:
        print(f"Error decoding string: {e}")
        return None

In [335]:
predicted_summaries = []

In [336]:
for sample_dialogue in test_dialogues:
  input = tokenizer(
    alpaca_prompt.format(
        instruction,
        sample_dialogue,
        ""
    ), return_tensors="pt"
  ).to("cuda")

  outputs = baseline_model.generate(
    **input,
    max_new_tokens=256,
    use_cache=True
  )

  predicted_summary = tokenizer.decode(outputs[0])

  output_str = extract_summary_from_string(predicted_summary)

  predicted_summaries.append(output_str)

###*Evaluate*

###**(B) Evaluate Performance of Base Model**

In [337]:
bert_scorer = evaluate.load("bertscore")

In [338]:
# Provide Prediction Summaries and Test Summaries as input
score = bert_scorer.compute(
    predictions= predicted_summaries,
    references= test_summaries,
    lang="en",
    rescale_with_baseline=True
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [339]:
#  Calculate Average Bert Score . Average Bert Score is sum of f1 score divided by number of samples
average_bert_score = np.mean(score['f1'])

In [342]:
average_bert_score

0.3886534199118614

###*Llama 2 Trained (Fine-tuned) Model Summarizer*

In [351]:
lora_model_name = "dialogue-summarizer-llama"

In [352]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [353]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [354]:
!cp -r /content/drive/MyDrive/{lora_model_name} .

###**(C) Load Trained Model**

In [355]:
import gc

# Function to clear GPU memory
def clear_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

clear_gpu_memory()


In [357]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 17301504.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [358]:
FastLanguageModel.for_inference(model)

In [359]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

###*Single Inference*

In [360]:
torch.cuda.empty_cache()

In [361]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [362]:
input = tokenizer(
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        ""
    ), return_tensors="pt"
).to("cuda")

In [363]:
output = model.generate(
    **input,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [364]:
print(tokenizer.decode(output[0]))

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

###*Batch Inference*

In [365]:
torch.cuda.empty_cache()

In [366]:
instruction = "Summarize the following dialogue"

In [ ]:
# test_size = 4

In [367]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

In [368]:
def extract_summary_from_string(input_string):
    try:
        # Assuming the response is between ### Response: and </s>
        summary_start = input_string.rfind('### Response:\n') + 14 # number of characters in '### Response:\n'
        summary_end = input_string.rfind('</s>')
        summary_str = input_string[summary_start:summary_end]

        return summary_str
    except Exception as e:
        print(f"Error decoding string: {e}")
        return None

In [369]:
predicted_summaries = []

In [370]:
for sample_dialogue in test_dialogues:
  input = tokenizer(
    alpaca_prompt.format(
        instruction,
        sample_dialogue,
        ""
    ), return_tensors="pt"
  ).to("cuda")

  outputs = model.generate(
    **input,
    max_new_tokens=256,
    use_cache=True
  )

  predicted_summary = tokenizer.decode(outputs[0])

  output_str = extract_summary_from_string(predicted_summary)

  predicted_summaries.append(output_str)

###*Evaluate*

###**(D) Evaluate Performance of Trained Model**

In [371]:
# Input prediction summaries and test summaries in bert scorer
score = bert_scorer.compute(
    predictions=predicted_summaries,
    references=test_summaries,
    lang="en",
    rescale_with_baseline=True
)

In [372]:
#  Calculate Average Bert Score . Average Bert Score is sum of f1 score divided by number of samples
average_bert_score = np.mean(score['f1'])

In [373]:
average_bert_score

0.4005774036049843

##**Question 14: Observations, Insights and Business Recommendations (20 Marks)**